## tfkeras でモデルの保存と復元を試す
- Reference
  - https://www.tensorflow.org/tutorials/keras/save_and_load?hl=ja

In [2]:
#!pip install -q h5py pyyaml 

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.0.0'

In [6]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

print(train_images.shape)

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

print(train_images.shape)

(60000, 28, 28)
(1000, 784)


In [7]:
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam', 
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model


model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [8]:
checkpoint_path = "data/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images,
          train_labels,
          epochs = 10, 
          validation_data = (test_images, test_labels),
          callbacks = [cp_callback])

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 672/1000 [===================>..........] - ETA: 0s - loss: 1.4153 - accuracy: 0.6057 
Epoch 00001: saving model to data/training_1/cp.ckpt
1000/1000 [==============================] - 1s 807us/sample - loss: 1.1869 - accuracy: 0.6660 - val_loss: 0.7677 - val_accuracy: 0.7490
Epoch 2/10
 608/1000 [=================>............] - ETA: 0s - loss: 0.4849 - accuracy: 0.8618
Epoch 00002: saving model to data/training_1/cp.ckpt
1000/1000 [==============================] - 0s 164us/sample - loss: 0.4405 - accuracy: 0.8770 - val_loss: 0.5247 - val_accuracy: 0.8330
Epoch 3/10
 672/1000 [===================>..........] - ETA: 0s - loss: 0.2819 - accuracy: 0.9315
Epoch 00003: saving model to data/training_1/cp.ckpt
1000/1000 [==============================] - 0s 155us/sample - loss: 0.2933 - accuracy: 0.9250 - val_loss: 0.4760 - val_accuracy: 0.8530
Epoch 4/10
 640/1000 [==================>...........] - ETA: 0s - loss: 0.2223 - accura

In [9]:
model = create_model()

loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 2.3838 - accuracy: 0.1680
Untrained model, accuracy: 16.80%


In [10]:
model.load_weights(checkpoint_path)

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.4808 - accuracy: 0.8740
Restored model, accuracy: 87.40%


### オプション